In [ ]:
# Module 17 Deliverable 1

# install pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-12-19 01:21:35--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2022-12-19 01:21:35 (11.4 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
# Read in the Review dataset as a DataFrame
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...|2015-08-31 00:00:00|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...| 

In [ ]:
# Drop null values
dropna_df = df.dropna()
dropna_df.show(10)
     

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...|2015-08-31 00:00:00|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...| 

In [ ]:

# Load in a sql function to use columns
from pyspark.sql.functions import col

# Drop duplicates
clean_df = dropna_df.drop_duplicates()
clean_df.show(10)
     

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   20009611|R100E0YCS06YNR|B00EFFW0HC|     277685961|       Battlefield 4|     Video Games|          5|            0|          0|   N|                Y|          Five Stars|     great thank you|2015-07-04 00:00:00|
|         US|   21036809|R100HR44BSB9I4|B007FTE2VW|     265303108|SimCity - Limited...| 

### Create DataFrames to match tables

In [ ]:
# Create the customers_table DataFrame
customers_df = clean_df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show(20)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   16505131|             1|
|   17235913|             1|
|   17782326|             2|
|   40985731|             7|
|   31554643|             1|
|   47501193|             7|
|    6367756|            65|
|   45401845|             1|
|   42066493|             8|
|    5483518|             1|
|   45302446|             8|
|   22728095|             1|
|   50093726|             1|
|   49336508|            21|
|   21442515|             4|
|   32582068|             1|
|   52193482|             2|
|    3172350|             1|
|   21340218|             8|
|   12550672|            10|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Create the products_table DataFrame and drop duplicates. 
products_df = clean_df.select(["product_id","product_title"]).drop_duplicates()
products_df.show(20)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00CJ7IUI6|The Elder Scrolls...|
|B00DHF39KS|Wolfenstein: The ...|
|B00MUTAVH6|Under Night In-Bi...|
|B001AZSEUW|              Peggle|
|B00KVOVBGM|PlayStation 4 Con...|
|B00O9VGH4Y|USPRO&reg; Headph...|
|B004OQNZY4|Phineas and Ferb:...|
|B00ZLN980O|Donop seablue 2.4...|
|B002L8W5V6|Dotop Nintendo Ga...|
|B007AJZ5PY|Nyko Game Case fo...|
|B000AOEU2K|Fire Emblem: Path...|
|B000H8BW7U|Tanarus (PC) (Com...|
|B013RADQOQ|Susenstone® 2400D...|
|B00KQXKUJ2|FIFA 15 (Ultimate...|
|B006W41X2C|Turtle Beach - Ea...|
|B000KCX9M4|Grand Theft Auto:...|
|B00YT90JWC|Red Wii Mini Cons...|
|B0096KG6A8|Wii U Super Mario...|
|B00L6AVLB0|World of Tanks-X3...|
|B000IMYKQ0|Wii Nunchuk Contr...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Create the review_id_table DataFrame. 
from pyspark.sql.functions import to_date
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")

review_id_df = clean_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show(20)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100E0YCS06YNR|   20009611|B00EFFW0HC|     277685961| 2015-07-04|
|R100HR44BSB9I4|   21036809|B007FTE2VW|     265303108| 2013-03-07|
|R100UABZOUF1JP|    4444018|B005HN5LKY|     148114874| 2015-08-17|
|R101MOW8A5WC4J|   51581032|B009LEIUY4|     563376217| 2014-08-16|
|R101X7KSBGIWF1|   41543132|B00004UC1W|     940154393| 2014-12-28|
|R101ZTSTDC8KQ4|    2074450|B006WN5UM2|     304022369| 2014-03-10|
|R103Q4U55JZAD1|   11407363|B0025KZV7A|     130147097| 2011-09-06|
|R103SW6BLR1HH6|   10547153|B004ACGO0Y|     540734971| 2011-09-25|
|R1048AZPZ1XQVY|   15338940|B005XTRC7M|     325268333| 2015-01-22|
|R104C56FZRE21Q|   49041788|B001US6O2Y|     855199089| 2015-05-20|
|R105REUM2343A3|   38221348|B0032GTMEE|     222888087| 2014-07-21|
|R106OW8SHRWO6E|   13133148|B00004S9AF|     132087124| 2007-08

In [ ]:
# Create the vine_table. DataFrame
vine_df = clean_df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R100E0YCS06YNR|          5|            0|          0|   N|                Y|
|R100HR44BSB9I4|          1|            1|          1|   N|                Y|
|R100UABZOUF1JP|          1|            0|          0|   N|                Y|
|R101MOW8A5WC4J|          5|            0|          0|   N|                Y|
|R101X7KSBGIWF1|          1|            0|          0|   N|                Y|
|R101ZTSTDC8KQ4|          5|            0|          0|   N|                Y|
|R103Q4U55JZAD1|          5|            0|          0|   N|                Y|
|R103SW6BLR1HH6|          5|            0|          0|   N|                Y|
|R1048AZPZ1XQVY|          4|            0|          0|   N|                Y|
|R104C56FZRE21Q|          5|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://:5432/Module_17_challenge"
config = {"user":"postgres", 
          "password": "", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)